In [1]:
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
!pwd

/home/btr-dev/wrkspc/prj/salty-hackers/ML-Model/Saltiest-Hackers-ML-Model


In [2]:
DIR = 'data/raw/gcp-bq-full/'
FILES = [str(i) for i in list(range(1, 18))]

In [3]:
df = pd.read_csv(DIR+FILES[0])

In [4]:
def refine(df):
    
    df = df.loc[df['type'] == 'comment']
    df = df.loc[df['by'].isnull() == False]
    df = df[['id', 'time', 'by', 'parent', 'text', 'type']]
    df['parent'] = df['parent'].astype(int)    
    return df

def scrub(doc):
    patt = {
            "unicode_patt": "&.{4}(?=;);",
            "line_break":   "<p>",
            "href_patt":    "<a.*</a>",
            "quote":        "&quot;",
            "html_footnote": "\[.\]",
            "punctuation":   "[^\w\s]",
            "numbers":       "[^A-Za-z\s]",
        }
    
    r = rf'|'.join(patt.values())
    return re.sub(r, ' ', str(doc))

def process_text(df):
    
    df.rename(columns={'text':'orig_text'})
    # regex
    df['cleaned_txt'] = df['orig_text'].apply(scrub)
    # lowercase
    df['cleaned-txt'] = df['cleaned_txt'].str.lower()
    # double spaces
    df['cleaned-txt'] = df['cleaned-txt'].str.replace('  ', ' ')
    
    
    
    # word freq
    word_freq = pd.Series(' '.join(df['text']).split()).value_counts()
    
    common = list(word_freq[:10].index)
    rare = list(word_freq[word_freq.values < 2].index)
    
    stop_words = list(nltk.corpus.stopwords.words('english'))    
    stop_words = set(stop_words + common + rare)
    
    print('removing stopwords')
    pat = r'\b(?:{})\b'.format('|'.join(stop_words))
    
    df['no_stopwords'] = df['text'].str.replace(pat, '')
    df['no_stopwords'] = df['no_stopwords'].str.replace(r'\s+', ' ')
     

    
    return df

(1319858, 14)


(1076413, 6)

In [12]:
stop_words = list(nltk.corpus.stopwords.words('english'))

df = refine(df)

In [6]:
df_test[df_test['text'].str.len() < 20]

,id,time,by,parent,text,type
158,10632127,1.448530e+09,eccstartup,10631668,I want to buy 10.,comment
247,22294726,1.581379e+09,sitkack,22292328,Sorry Helen.,comment
412,1948012,1.290973e+09,stevenj,1948002,Sent.,comment
460,14366695,1.495111e+09,fiatjaf,14366160,You&#x27;re wrong.,comment
500,19306722,1.551742e+09,trimstray,19306679,Sure. PRs welcome!,comment
...,...,...,...,...,...,...
1319577,21203725,1.570632e+09,malms,21193934,this is nice,comment
1319592,20076440,1.559493e+09,ehvatum,20076361,Stephen Elop,comment
1319623,567917,1.240019e+09,adharmad,567667,www.mew.org,comment
1319672,19933206,1.558041e+09,julianozen,19931977,[deleted],comment


In [26]:
test.rename(columns)
# test = process_text(test)

NameError: name 'columns' is not defined

In [23]:
for p in test['text'][:10]:
    print(p, '\n')

 incredibly toxic behavior mandated reply post downvoting hey let person participating  

 could everything right make mistakes still fail life point though economy core indirect representation host society morals values throwing around moral seem appropriate hard side right thing ie morally responsible carries risk endure  

 meant happened chicago elsewhere sf  

 black ones actually cloud iowa quite wind farms source pride maybe met  

 nothing setting rigid password policy give appearance security providing secondary super weak password ssn easy figure security questions many sites purely appearance little real thought given protecting users  

reminds high school experience kids top class worked hard appear work hard among relieved find students university genuinely cared worked hard academics ones failed kids future high school reputation difficult thrilled care c students want challenged  

 exactly taxed stock state grant sale value understand law  

seconded run two small webs

In [8]:
df.head()

,id,time,by,parent,text,type
0,4049595,1.338489e+09,cpeterso,4049139,I started to write a C++ template class that w...,comment
1,14633082,1.498437e+09,fweespeech,14632856,&gt; I&#x27;m guessing over $200B of Amazon&#x...,comment
2,19418238,1.552875e+09,chrisseaton,19418216,I don&#x27;t know what to say - that just soun...,comment
3,16130436,1.515732e+09,Aloha,16130413,if you&#x27;re going to make the accusation th...,comment
4,6589315,1.382406e+09,drill_sarge,6588825,With the current trend of simplifying your int...,comment


In [ ]:
stop_words = list(nltk.corpus.stopwords.words('english'))

for file in FILES:
    path = DIR + file
    tmp = pd.read_csv(file)
    tmp = refine(tmp)
    tmp = process_text(tmp)
    tmp.to_csv(f'processed/preproc-1/hn{file}')
    

## Thoughts

- lots of 1x use words
- lots of misspelled words being picked up
- storing cleaned original text may prove costly / all formatting we remove is html.


## Model stuff

 Transfer learning student -> teacher

[tutorial(https://adventuresinmachinelearning.com/transfer-learning-tensorflow-2/)

 Fine Tuning notebook from google research
https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb

 distilbert smaller model but close to same results
https://medium.com/huggingface/distilbert-8cf3380435b5
https://github.com/huggingface/transformers/tree/master/examples/distillation


 Bert fine grained sentiment classification
https://arxiv.org/pdf/1910.03474v1.pdf

 Interesting set to train on
https://www.aclweb.org/anthology/D19-5544.pdf

adverserial training with bert
https://export.arxiv.org/pdf/2001.11316





